In [1]:
import pandas as pd
import numpy as np
import statistics
import csv
from sklearn import metrics
from xgboost import XGBClassifier
import random
import math
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier

In [2]:
#Extracting The Spatial Feature

def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        #print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

# Chameleon

In [4]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/Wiki/Chameleons/out1_graph_edges.txt")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
#print(mylist)

'node_id\tnode_id'

In [5]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
#print(node_ID)
n=len(node_ID)
print(n)

2277


In [6]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/Wiki/Chameleons/out1_node_feature_label.txt',sep='\s+')
#df.to_csv('my_file.csv', header=None)
df.head()

,node_id,feature,label
0,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0
1,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
2,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
3,3,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",4
4,4,"0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...",2


In [8]:
fe=df["feature"].to_numpy()
lst_Class=df["label"].to_numpy()
lst_ID=df["node_id"].to_numpy()

lst_Fe=[]
for row in fe:
    d=row.split(',')
    fev=[]
    for j in range(len(d)):
            fev.append(int(d[j]))
    lst_Fe.append(fev)

In [9]:
fe_len=len(lst_Fe[0])
feature_names = ["w_{}".format(ii) for ii in range(fe_len)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=fe_len,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2316,w_2317,w_2318,w_2319,w_2320,w_2321,w_2322,w_2323,w_2324,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [10]:
Class=lst_Class.copy()
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2316,w_2317,w_2318,w_2319,w_2320,w_2321,w_2322,w_2323,w_2324,Class
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
#print(Basis_0)
count0=0
count1=0
for i in range(fe_len):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

1694
631


In [21]:
SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.01):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.01):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.01):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.01):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.01):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
#print(SBasis_3)
count0=0
count1=0
for i in range(fe_len):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

1875
450


In [23]:
Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
#print(Fec)

In [25]:
SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(fe_len):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
#print(SFec)

In [26]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,2,3,5,5,7
1,1,4,8,16,15
2,3,4,19,33,28
3,0,3,4,13,24
4,2,11,30,37,46


In [27]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,7,6,6,6,7
1,10,12,18,20,18
2,19,24,39,43,40
3,8,16,17,23,25
4,27,36,62,56,57


In [28]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    Node_class=[0,1,2,3,4,5]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Class)
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
    
    
    class_60=Class.copy()
    i=0
    for d in p:
        class_60[d]=y_pred[i]
        i=i+1
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]
    #print(A)
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=20
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [29]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 60.06578947368421
Standard Deviation: 1.8185453843859107
Domain only
Mean: 72.10526315789474
Standard Deviation: 2.2046473387303305
Accuracy using both
Mean: 77.76315789473684
Standard Deviation: 0.9486039881583772


# Squirrel

In [31]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/Wiki/Squirrel/out1_graph_edges.txt")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
#print(mylist)

'node_id\tnode_id'

In [32]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
#print(node_ID)
n=len(node_ID)
print(n)

5201


In [33]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
df = pd.read_csv('https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/Wiki/Squirrel/out1_node_feature_label.txt',sep='\s+')
#df.to_csv('my_file.csv', header=None)
df.head()

,node_id,feature,label
0,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1
1,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
2,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0
3,3,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0
4,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",4


In [35]:
fe=df["feature"].to_numpy()
lst_Class=df["label"].to_numpy()
lst_ID=df["node_id"].to_numpy()

lst_Fe=[]
for row in fe:
    d=row.split(',')
    fev=[]
    for j in range(len(d)):
            fev.append(int(d[j]))
    lst_Fe.append(fev)

In [36]:
fe_len=len(lst_Fe[0])
feature_names = ["w_{}".format(ii) for ii in range(fe_len)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=fe_len,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2080,w_2081,w_2082,w_2083,w_2084,w_2085,w_2086,w_2087,w_2088,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [37]:
Class=lst_Class.copy()
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_2080,w_2081,w_2082,w_2083,w_2084,w_2085,w_2086,w_2087,w_2088,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [38]:
Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
#print(Basis_0)
count0=0
count1=0
for i in range(fe_len):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

828
1261


In [39]:
SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.01):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.01):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.01):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.01):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.01):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

count0=0
count1=0
for i in range(fe_len):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

1412
677


In [40]:
Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
#print(Fec)

In [41]:
SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(fe_len):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
#print(SFec)

In [42]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,21,27,30,34,35
1,11,19,22,28,27
2,1,1,1,1,1
3,0,2,2,2,2
4,7,11,16,18,20


In [43]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,35,37,37,37,37
1,29,32,32,32,32
2,1,1,1,1,1
3,2,2,2,2,2
4,20,20,20,20,20


In [48]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

In [49]:
for it in range(5):
    print("\rProcessing file {} ({}%)".format(it, 100*it//(n-1)), end='', flush=True)
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    Node_class=[0,1,2,3,4,5]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Class)
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    bst = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1)
    bst.fit(X_train,y_train)
    y_pred=bst.predict(X_test)
    
    
    class_60=Class.copy()
    i=0
    for d in p:
        class_60[d]=y_pred[i]
        i=i+1
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]
    #print(A)
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    
    bst = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1)
    bst.fit(X_train,y_train)
    y_pred=bst.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=len(F_vec[0])
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    bst = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1)
    bst.fit(X_train,y_train)
    y_pred=bst.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    bst = XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1)
    bst.fit(X_train,y_train)
    y_pred=bst.predict(X_test)
    
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

Processing file 4 (0%)

In [47]:
mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial Only
Mean: 41.95965417867435
Standard Deviation: 1.8218788298898783
Domain only
Mean: 41.671469740634
Standard Deviation: 1.487247636772653
Accuracy using both
Mean: 52.23823246878002
Standard Deviation: 1.2215304022295357
